# On the Virasoro fusion kernel at $c=25$

This notebook provides numerical tests for some of the main claims of the article with the same title, by Sylvain Ribault and Ioannis Tsiares.

## Tetrahedral notation

$$
\begin{array}{|l|c|l|}
  \hline 
  \text{Name} & \text{Notation} & \text{Value}
  \\
  \hline\hline 
  \text{Edges} & E & \{1,2,3,4,s,t\} 
  \\
  \text{Pairs of opposite edges} & P & \{13,24,st\}
  \\
  \text{Faces} & F & \{12s,34s,23t,14t\}
  \\
  \text{Vertices} & V &  \{14s,12t,34t,23s\}
  \\
  \hline 
 \end{array}
 $$
 
* $\sigma\in\mathbb{Z}_2^E$ is an assignment of a sign $\sigma_i\in\{+,-\}$ for any $i\in E$, and $\sigma\in \mathbb{Z}_2^f$ for a triple of signs on a face $f\in F$.
* $\sigma_E,\sigma_v,\sigma_f,\sigma_p$ for products of $6,3,3$ or $2$ signs on all edges, a vertex, a face, or two opposite edges.
* $\sigma\in\mathbb{Z}_2^E|\sigma_V=1$ for sign assignments whose products are $1$ at each vertex. There are $8$ such assignments, and they can be split in two halves according to $\sigma_E=\pm 1$.
* The indicator function $\eta_i\in \mathbb{Z}_2^F$ is $\eta_i(f)=1$ if the edge $i$ belongs to the face $f$, and $\eta_i(f)=-1$ otherwise. 

In [1]:
class tetra:
    
    edges = frozenset(['1', '2', '3', '4', 's', 't'])
    faces = frozenset([frozenset(['1', '2', 's']), frozenset(['3', '4', 's']), 
                       frozenset(['2', '3', 't']), frozenset(['1', '4', 't'])])
    vertices = frozenset([frozenset(['1', '2', 't']), frozenset(['3', '4', 't']), 
                       frozenset(['2', '3', 's']), frozenset(['1', '4', 's'])])
    pairs = frozenset([frozenset(['s', 't']), frozenset(['1', '3']), frozenset(['2', '4'])])
    
    def all_signs(aset):
        """ Assigns a sign to each element of a set of edges. """
        signs = [dict()]
        for edge in aset: 
            newsigns = []
            for sign in signs:
                sign2 = sign.copy()
                sign[edge] = 1
                sign2[edge] = -1
                newsigns += [sign, sign2]
            signs = newsigns 
        return signs
    
    def condition(signs, sets_edges):
        """ signs = a list of signs on a set of edges.
            edges = a list of sets of edges: we want the product of signs over each set to be one. """
        signs1 = signs.copy()
        for set_edges in sets_edges:
            to_remove = []
            for sign in signs1:
                p = 1
                for edge in set_edges: p *= sign[edge]
                if p == -1: to_remove.append(sign)
            for sign in to_remove: signs1.remove(sign)     
        return signs1
    
    def six_signs():
        """ Assigns a sign to each edge such that products of signs at vertices are 1. 
            Returns 8 list of 7 signs, the last one being the product. """
        signs = tetra.condition(tetra.all_signs(tetra.edges), tetra.vertices)
        for sign in signs: 
            p = 1
            for edge in tetra.edges: p *= sign[edge]
            sign['prod'] = p
        return signs
    
    def three_signs(face, edge):
        """ Assigns a sign to each edge of a face, such that the product of signs is 1 if 
        the face contains the edge, -1 else. This product is appended to the signs. 
        """
        signs1 = tetra.all_signs(face)
        signs2 = tetra.condition(signs1, [face])
        if edge in face: 
            for sign in signs2: sign['prod'] = 1
            return signs2
        else: 
            signs = [sign for sign in signs1 if not sign in signs2]
            for sign in signs: sign['prod'] = -1
            return signs

In [2]:
""" Some basic tests of the tetrahedral notation. """

print(''.join(tetra.edges))

for face in tetra.faces:
    print(''.join(face), [[val for val in sign.values()] for sign in tetra.three_signs(face, 't')])
    # print(tetra.three_signs(face, 't'))

print(tetra.six_signs());

4ts132
3s4 [[1, 1, -1, -1], [1, -1, 1, -1], [-1, 1, 1, -1], [-1, -1, -1, -1]]
1t4 [[1, 1, 1, 1], [1, -1, -1, 1], [-1, 1, -1, 1], [-1, -1, 1, 1]]
2s1 [[1, 1, -1, -1], [1, -1, 1, -1], [-1, 1, 1, -1], [-1, -1, -1, -1]]
32t [[1, 1, 1, 1], [1, -1, -1, 1], [-1, 1, -1, 1], [-1, -1, 1, 1]]
[{'4': 1, 't': 1, 's': 1, '1': 1, '3': 1, '2': 1, 'prod': 1}, {'4': 1, 't': 1, 's': -1, '1': -1, '3': 1, '2': -1, 'prod': -1}, {'4': 1, 't': -1, 's': 1, '1': 1, '3': -1, '2': -1, 'prod': -1}, {'4': 1, 't': -1, 's': -1, '1': -1, '3': -1, '2': 1, 'prod': 1}, {'4': -1, 't': 1, 's': 1, '1': -1, '3': -1, '2': -1, 'prod': 1}, {'4': -1, 't': 1, 's': -1, '1': 1, '3': -1, '2': 1, 'prod': -1}, {'4': -1, 't': -1, 's': 1, '1': -1, '3': 1, '2': 1, 'prod': -1}, {'4': -1, 't': -1, 's': -1, '1': 1, '3': 1, '2': -1, 'prod': 1}]


## Kernels

With $G$ the Barnes $G$-function, we introduce the notations:
$$
\begin{align}
\widetilde{G}(x) &= \frac{G(1+x)}{G(1-x)}
\\
 d &= \det \begin{pmatrix}
1 & -\cos{(2\pi P_2)} & -\cos{(2\pi P_3)} & \cos{(2\pi P_s)} \\
-\cos{(2\pi P_2)} & 1  & \cos{(2\pi P_t)} & -\cos{(2\pi P_1)} \\
-\cos{(2\pi P_3)} & \cos{(2\pi P_t)} & 1  & -\cos{(2\pi P_4)} \\
\cos{(2\pi P_s)} & -\cos{(2\pi P_1)} & -\cos{(2\pi P_4)} & 1  \\
\end{pmatrix}
\\
e^{2\pi i\omega_\pm} &= -\frac{\sum_{ij\in P} \sin{(2\pi P_i)}\sin{(2\pi P_j)} \pm \sqrt{d}}{\frac{1}{2}\sum_{\substack{\sigma\in\mathbb{Z}_2^E \\ \sigma_V = 1}} \sigma_E e^{\pi i\sum_{i\in E}\sigma_iP_i}} 
\end{align}
$$
With these notations we compute the quantities:
$$
\begin{align}
\text{Prefactor } &= \frac{\pi^2}{i}
 \frac{G(\pm 2P_t)}{G(2\pm 2P_s)} 
 \prod_{f\in F} \prod_{\substack{\sigma\in \mathbb{Z}_2^f \\ \sigma_f=\eta_t(f)}} G\left(1+\textstyle{\sum}_{j\in f}\sigma_jP_j\right)^{-\sigma_f}
\\
\text{Products}(\epsilon) &= \frac{1}{\sqrt{d}} 
 \prod_{\substack{\sigma\in\mathbb{Z}_2^E\\ \sigma_V = 1}}\widetilde{G}\left(\omega_\epsilon-\tfrac12\textstyle{\sum}_{i\in E}\sigma_iP_i\right)^{-\sigma_E}
\\
\text{Integral } &= 
\int_{i\mathbb{R}} du \prod_{\substack{\sigma\in\mathbb{Z}_2^E\\ \sigma_V = 1}} \widetilde{G}\left(u+\tfrac{\sigma_E}{2}+\tfrac12\textstyle{\sum}_{i\in E} \sigma_i P_i\right)^{\sigma_E}
\end{align}
$$
We then do the following calculations:

* Check the identity:
$$
4\pi^4\Big(\text{Products}(+) - \text{Products}(-)\Big) = \text{Integral} 
$$
* Compute test values of the $c=25$ kernels:
$$
\mathbf{F}^{\pm} := \pm \text{ Prefactor } \times \text{Products}(\pm) \ , \ \mathbf{F} = \frac12 \Big(\mathbf{F}^{+}+\mathbf{F}^{-}\Big) 
$$
* Check consistency with special case of $P_{ext}=\tfrac14$, where:
$$
\mathbf{F}^{\pm} = \pm i\frac{P_t}{P_s} 16^{P_t^2-P_s^2} e^{\pm 2\pi i P_s P_t} 
$$
* Test parity and permutations symmetries where we observe the following:
    - Reflections in $P_s,P_t$: $\mathbf{F}^{\pm}\rightarrow \mathbf{F}^{\mp}$ 
    - Reflections in $P_i$, $i=1,2,3,4$:  $\mathbf{F}^{\pm}\rightarrow \mathbf{F}^{\pm}$
    - Certain permutations of $P_i$, $i=1,2,3,4$: $\mathbf{F}^{\pm}\rightarrow \mathbf{F}^{\pm}$
* Check the fusion transformation:
$$ 
\mathcal{F}^{(b=1),\text{$s-$channel}}_{P_s} = \int_{i\mathbb{R}}\frac{dP_t}{i} \ \mathbf{F}_{P_s,P_t} \mathcal{F}^{(b=1),\text{$t-$channel}}_{P_t}
$$
To compute the conformal blocks, we import code from the repository https://gitlab.com/s.g.ribault/Bootstrap_Virasoro

In [3]:
from mpmath import cos, sin, pi, mp, log, sqrt, exp, barnesg, quad, inf
import time

def tG(x):
    return barnesg(1+x)/barnesg(1-x)
def c(x):
    return cos(2*pi*x)
def s(x):
    return sin(2*pi*x)

class kernel:
    
    def __init__(self, moms):
        """ moms = dictionary of six momentums. """
        
        self.moms = moms
        self.cs = {e: c(P) for e, P in moms.items()}
        self.ss = {e: s(P) for e, P in moms.items()}
        prec = mp.dps
        mp.dps = 2*prec   
        """ Temporary increase in precision to avoid undetermined omega, due to a numerator 
        that vanishes if its two terms are dominated by the same large momentum. """
        self.sqrt_d = sqrt(self.dDet()) if (s(moms['s'])*s(moms['t'])).real>0 else -sqrt(self.dDet())
        # Choosing the correct branch of the square root.
        self.oms = self.omegas()
        mp.dps = prec
        self.prods = self.products()
        self.pref = self.prefactor()
        self.valuep = self.pref*self.prods[0]              # The + kernel.
        self.valuem = -self.pref*self.prods[1]             # The - kernel.
        self.value = (self.valuep + self.valuem)/2         # The kernel.
        
    @staticmethod
    def prod(values, edges):
        """ Product of some values over a set of edges. """
        result = 1
        for edge in edges: result *= values[edge]
        return result
        
    def dDet(self):

        moms = self.moms
        dmatrix = mp.matrix([[1, -c(moms['2']), -c(moms['3']), c(moms['s'])], 
                            [-c(moms['2']), 1, c(moms['t']), -c(moms['1'])], 
                            [-c(moms['3']), c(moms['t']), 1, -c(moms['4'])], 
                            [c(moms['s']), -c(moms['1']), -c(moms['4']), 1]])
        result = mp.det(dmatrix)
        return result
        
    def omegas(self):
        
        num = sum(kernel.prod(self.ss, pair) for pair in tetra.pairs) 
        den = 1/2*sum(signs['prod']*exp(1j*pi*sum(signs[edge] * self.moms[edge] 
                                                  for edge in tetra.edges)) for signs in tetra.six_signs())
        return [log(-(num + sign*self.sqrt_d)/den)/2/pi/1j for sign in [1, -1]]
        
    def prefactor(self):
        
        result = pi**2/1j 
        moms = self.moms
        for sign in [-1, 1]: result *= barnesg(sign*2*moms['t'])/barnesg(2+sign*2*moms['s'])
        for face in tetra.faces:
            for signs in tetra.three_signs(face, 't'): 
                result *= barnesg(1 + sum(signs[edge]*moms[edge] 
                                          for edge in face))**(-signs['prod'])
        return result
    
    def products(self):
        
        results = []
        for omega in self.oms:
            result = 1/self.sqrt_d
            for signs in tetra.six_signs():
                result *= tG(omega - 1/2*sum(signs[edge]*self.moms[edge] 
                                             for edge in tetra.edges))**(-signs['prod']) 
            results.append(result)
        return results
    
    def integral(self, cutoff = inf):
        
        def integrand(u):
            result = 1
            for signs in tetra.six_signs():
                result *= tG(1j*u + signs['prod']/2 + 1/2*sum(signs[edge]*self.moms[edge] 
                                             for edge in tetra.edges))**(signs['prod'])
            return 1j * result
        
        return quad(integrand, (-cutoff, cutoff))

In [4]:
""" Testing the agreement between the product and integral formulas. """ 

precs = [10, 20, 40]
integrals = []
products = []
for prec in precs: 
    mp.dps = prec
    print('mp.dps =', prec)
    moms1 = {'1': mp.mpc('.01', '.32'), '2': mp.mpc('.03', '.41'), '3': mp.mpc('.07', '.49'), 
         '4': mp.mpc('.11', '.12'), 's': mp.mpc('.06', '.24'), 't': mp.mpc('.14', '.72')}
    ker = kernel(moms1)
    time0 = time.time()
    integral = ker.integral()
    print("Time:", "%.4g" % (time.time() - time0))
    integrals.append(integral)
    prods = ker.products()
    x = (prods[0] - prods[1])*4*pi**4
    print("Value of the product:", x)
    products.append(x)
    print("Relative difference:", "%.4g" % abs(1-integral/x))

mp.dps = 10
Time: 10.45
Value of the product: (0.6889133867 + 0.6441105021j)
Relative difference: 1.552e-10
mp.dps = 20
Time: 36
Value of the product: (0.68891338682666915949 + 0.64411050206768483528j)
Relative difference: 6.25e-21
mp.dps = 40
Time: 162.7
Value of the product: (0.6889133868266691594987524999321416913645 + 0.6441105020676848352864149133016713851826j)
Relative difference: 1.021e-40


In [5]:
""" Computing the kernels. """ 

mp.dps = 30
moms1 = {'1': mp.mpc('.01', '.32'), '2': mp.mpc('.03', '.41'), '3': mp.mpc('.07', '.49'), 
         '4': mp.mpc('.11', '.12'), 's': mp.mpc('.06', '.24'), 't': mp.mpc('.14', '.72')}
ker = kernel(moms1)
print(ker.valuep)
print(ker.valuem)
print(ker.value)

(-0.154097981703016855475058795052 + 3.27853981340746619623343190369j)
(1.60927018914193406359441027039 - 3.74571522809768372496448782048j)
(0.727586103719458604059675737668 - 0.233587707345108764365527958395j)


In [6]:
""" Checking Zamolodchikov's special cases. """

mp.dps = 30
momsZam = {'1': mp.mpc('.25', '0'), '2': mp.mpc('.25', '0'), '3': mp.mpc('.25', '0'), 
           '4': mp.mpc('.25', '0'), 's': mp.mpc('-.21', '.14'), 't': mp.mpc('.33', '.14')}
kerZam = kernel(momsZam)

def ZamFp(Ps,Pt): return (1j)*(Pt/Ps)*(16)**(Pt**2-Ps**2)*exp(2*pi*1j*Pt*Ps)
def ZamFm(Ps,Pt): return (1j)*(-Pt/Ps)*(16)**(Pt**2-Ps**2)*exp(-2*pi*1j*Pt*Ps) 
def ZamF(Ps,Pt): return (1/2)*(ZamFp(Ps,Pt) + ZamFm(Ps,Pt))
    
gp = ZamFp(momsZam['s'],momsZam['t'])
gm = ZamFm(momsZam['s'],momsZam['t'])
g = ZamF(momsZam['s'],momsZam['t'])

print("%.4g" % abs(1-kerZam.valuep/gp))
print("%.4g" % abs(1-kerZam.valuem/gm))
print("%.4g" % abs(1-kerZam.value/g))

9.878e-31
1.105e-30
6.974e-31


In [7]:
""" Parity and reflection symmetries. """

mp.dps = 30
Ps = {'1': mp.mpc('.01', '.32'), '2': mp.mpc('.03', '.41'), '3': mp.mpc('.07', '.49'), '4': mp.mpc('.11', '.12'), 
          's': mp.mpc('0.12','-.63'), 't': mp.mpc('0.07','-.37')}
val1 = kernel(Ps).valuem
Ps['t'] = -Ps['t']    # Reflecting the t-channel momentum
val2 = kernel(Ps).valuep
print("%.4g" % abs(1-val1/val2))
Ps = {'1': Ps['2'], '2': Ps['1'], '3': Ps['4'], '4': Ps['3'], 's': Ps['s'], 't': Ps['t']}
# Permuting 1 <-> 2 and 3 <-> 4
val3 = kernel(Ps).valuep
print("%.4g" % abs(1-val3/val2))
Ps = {'1': Ps['4'], '2': Ps['3'], '3': Ps['2'], '4': Ps['1'], 's': Ps['s'], 't': Ps['t']}
# Permuting 1 <-> 4 and 2 <-> 3
val4 = kernel(Ps).valuep
print("%.4g" % abs(1-val4/val3))
Ps['3'] = -Ps['3']   # Reflecting an external momentum
val5 = kernel(Ps).valuep
print("%.4g" % abs(1-val5/val4))

6.299e-31
9.919e-31
2.063e-31
8.336e-31


In [8]:
""" Checking fusion. This is where we need code from 
https://gitlab.com/s.g.ribault/Bootstrap_Virasoro . """

import sys
sys.path.append('/Users/Yiannis Tsiares/Desktop/Bootstrap_Vir') 
sys.path.append('/home/sylvain/repos/Bootstrap_Vir')
from Blocks import Block
from CFT import *
import time

for prec in [16, 32, 48, 64]:
    
        N = int(prec/4)
        print('dps =', prec, 'N =', N)
        mp.dps = prec
        Nmax = int(.6 * mp.dps)

        # z = mp.mpc('0.23','0.01')
        z = mp.mpc('.45', '.14')
        moms1 = {'1': mp.mpc('.0041', '-.031'), '2': mp.mpc('.0093', '.042'), '3': mp.mpc('.0039', '.047'),
                '4': mp.mpc('.0085', '-.035'), 's': mp.mpc('.0067', '.024')}
        Pext = [moms1['1'], moms1['2'], moms1['3'], moms1['4']]
        epsilon = 10**(-mp.dps/8)
        eps = [epsilon*exp(1j*pi/N*(2*k+1)) for k in range(N)]
        sblocks = []
        tblocks = []
        for k in range(N):
            cc = Charge('b', 1+eps[k])
            dims = {key: Dimension('P', val, cc) for key, val in moms1.items()}
            dims_ext = [dims['1'], dims['2'], dims['3'], dims['4']]
            sblocks.append(Block(dims_ext, Nmax = Nmax).value(position = z)(dims['s']))
            tblock = Block(dims_ext, Nmax = Nmax, channel = 't').value(position = z)
            def tblk(mom):
                return tblock(Dimension('P', mom, cc))
            tblocks.append(tblk)

        sblock = sum(sblocks)/N
        def integrand(y):
            momst = moms1.copy()
            momst['t'] = 1j * y
            ker = kernel(momst).value  
            bl = sum([blk(1j * y) for blk in tblocks])/N
            return ker * bl
        time0 = time.time()    
        integral = 2*quad(integrand, (0, 7.15))
        print("T-channel integral time:", "%.4g" % (time.time() - time0))
        print("S-channel block value:",sblock)
        print("Relative difference:","%.4g" % abs(1-integral/sblock))

dps = 16 N = 4
T-channel integral time: 17.66
S-channel block value: (2.803019954715486 - 0.4762027195696809j)
Relative difference: 9.559e-05
dps = 32 N = 8
T-channel integral time: 78
S-channel block value: (2.8030200043920556292880698387422 - 0.47620273906860037983674864366177j)
Relative difference: 9.625e-09
dps = 48 N = 12
T-channel integral time: 144.1
S-channel block value: (2.80302000439205562928806983873582795764653619253 - 0.476202739068600379836748643634534982150443982489j)
Relative difference: 9.626e-13
dps = 64 N = 16
T-channel integral time: 488.5
S-channel block value: (2.80302000439205562928806984162483412738814839306867251617938576 - 0.4762027390686003798367486443082769761216192150032420913369004316j)
Relative difference: 9.626e-17
